In [2]:
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import os 
from dotenv import load_dotenv


In [4]:
load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')

In [5]:
loader = CSVLoader("data.csv")
data=loader.load()

In [6]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_26412\909259017.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")


In [8]:
vector_db = FAISS.from_documents(data,embeddings)

In [9]:
vector_db.save_local('faiss_index')

In [12]:
rst=vector_db.similarity_search('Corn Leaf Miner Corn Medium')
rst[1].page_content

'Pest Name: Corn Stalk Borer\nCrop Type: Corn\nTreatment Recommendations: Cultural Control: Destroy crop residues, deep ploughing after harvest. Biological Control: Use Trichogramma egg parasitoids and predatory bugs. Chemical Control: Apply carbofuran granules or spray chlorantraniliprole during early larval stages. Physical Control: Use pheromone traps for monitoring.'

In [13]:
retriever= vector_db.as_retriever()

In [15]:
retriever.invoke("corn")

[Document(id='b192416f-3c0c-46a9-8246-3026e9de250b', metadata={'source': 'data.csv', 'row': 14}, page_content='Pest Name: Corn Stalk Borer\nCrop Type: Corn\nTreatment Recommendations: Cultural Control: Destroy crop residues, deep ploughing after harvest. Biological Control: Use Trichogramma egg parasitoids and predatory bugs. Chemical Control: Apply carbofuran granules or spray chlorantraniliprole during early larval stages. Physical Control: Use pheromone traps for monitoring.'),
 Document(id='711b3b2e-b159-4cdb-99b1-8ccbbf6705ee', metadata={'source': 'data.csv', 'row': 10}, page_content='Pest Name: Corn Earworm\nCrop Type: Corn\nTreatment Recommendations: Cultural Control: Plant early maturing varieties, destroy crop residues. Biological Control: Release Trichogramma wasps, encourage predatory beetles. Chemical Control: Apply spinosad, indoxacarb, or Bt-based insecticides at silk emergence. Monitoring: Use pheromone traps for adult monitoring.'),
 Document(id='71750bc9-6a48-47ca-a086

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_core.documents import Document

In [17]:
llm = ChatGroq(api_key=groq_api_key, model='Gemma2-9b-It')

In [18]:
prompt = ChatPromptTemplate.from_messages([
    ("system","You are an Treatement reccomender agent for a smart farming system," 
     "use the context below to provide accurate and precise treatement for the relevant pest and crop on the user input."
     "provide the user with a detailed description of the treatment."),
     ("system","<context>\n{context}\n</context>"),
     ("human", "Question:{input} and {croptype}")
])

In [19]:
doc_chain = create_stuff_documents_chain(llm,prompt)

In [20]:
chain = create_retrieval_chain(retriever,doc_chain)

In [21]:
result = chain.invoke({    "input": "Green Rice Leafhopper", "croptype": "Rice" })
result['answer']

"You're dealing with **Green Rice Leafhoppers** on your **rice** crop. \n\nHere's a breakdown of recommended treatments, prioritizing the most environmentally friendly options first:\n\n**1. Cultural Control:**\n\n* **Remove grassy weeds:** Green Rice Leafhoppers love to hang out in weeds, so keeping your field clean will significantly reduce their population.\n* **Maintain field sanitation:**  This means properly removing crop residue after harvest to eliminate overwintering sites.\n\n**2. Biological Control:**\n\n* **Conserve natural enemies:** Encourage spiders and other predators in your field by providing habitat and avoiding broad-spectrum pesticides that harm beneficial insects.\n\n**3. Physical Control:**\n\n* **Use yellow sticky traps:** These traps are effective for monitoring Green Rice Leafhopper populations and can also help control them by trapping adults. Place them throughout your field.\n\n**4. Chemical Control (Use as a last resort):**\n\n* **Apply systemic insecticid